In [1]:
import pandas as pd
import numpy as np
import pickle as pkl

In [38]:
# download dfs from local source
accidentDF = pd.read_csv('2015-traffic-fatalities/accident.csv', delimiter = ',')
vehicleDF = pd.read_excel('2015-traffic-fatalities/vehicleDF.xls')

In [39]:
# merge dfs - each contain individual case number which can be merged on
df = pd.merge(accidentDF, vehicleDF, on='ST_CASE')

In [40]:
df.head()

,STATE_x,ST_CASE,VE_TOTAL,VE_FORMS_x,PVH_INVL,PEDS,PERNOTMVIT,PERMVIT,PERSONS,COUNTY,...,VTRAFCON,VTCONT_F,P_CRASH1,P_CRASH2,P_CRASH3,PCRASH4,PCRASH5,ACC_TYPE,DEATHS,DR_DRINK
0,1,10001,1,1,0,0,0,1,1,127,...,0,0,14,13,99,1,4,1,1,1
1,1,10002,1,1,0,0,0,1,1,83,...,0,0,14,6,1,4,4,7,1,0
2,1,10003,1,1,0,0,0,2,2,11,...,0,0,1,13,1,1,4,1,1,1
3,1,10004,1,1,0,0,0,1,1,45,...,0,0,14,13,1,1,4,1,1,1
4,1,10005,2,2,0,0,0,2,2,45,...,0,0,11,15,1,1,1,68,1,0


In [41]:
# pick features I think can be indicative of target
df = df[['DR_DRINK', 'HOUR_x', 'VE_FORMS_x', 'VE_TOTAL', 'PERSONS', 'PEDS', 'NHS', 'HIT_RUN', 'ROLLOVER', 'TRAV_SP', 'VSPD_LIM', 'PREV_ACC', 'PREV_SUS', 'PREV_SPD', 'PREV_OTH']]
df.head()

,DR_DRINK,HOUR_x,VE_FORMS_x,VE_TOTAL,PERSONS,PEDS,NHS,HIT_RUN,ROLLOVER,TRAV_SP,VSPD_LIM,PREV_ACC,PREV_SUS,PREV_SPD,PREV_OTH
0,1,2,1,1,1,0,0,0,0,55,55,1,0,1,0
1,0,22,1,1,1,0,1,0,1,70,70,2,2,0,0
2,1,1,1,1,2,0,0,0,1,80,55,2,0,1,0
3,1,0,1,1,1,0,0,0,0,75,55,0,0,0,0
4,0,7,2,2,2,0,0,0,1,15,65,0,0,0,0


In [42]:
# clean up data (remove numbers representing unknowns)
df = df[(df.NHS != 9)]
df = df[(df.HIT_RUN != 9)]
df = df[(df.ROLLOVER != 9) & (df.ROLLOVER != 2)]
df = df[(df.TRAV_SP != 0) & (df.TRAV_SP < 151)]
df = df[(df.VSPD_LIM != 0) & (df.TRAV_SP < 98)]
df = df[(df.PREV_ACC < 10)]

In [43]:
# create new dummy variable for if the crash occurs between the hours of 7 PM and 7 AM and
# if the crash is a single vehicle crash
df['BETWEEN_7PM_AND_7AM'] = np.where((df['HOUR_x'] < 7) | (df['HOUR_x'] > 18), 1, 0)
df['SINGLE_VEHICLE_CRASH'] = np.where(df['VE_TOTAL'] == 1, 1, 0)

In [44]:
# final edits to remove unnecessary columns and rename others
df.rename(columns = {'VE_FORMS_x':'IN_TRANSPORT_VEHICLES_INVOLVED', 'VE_TOTAL':'TOTAL_VEHICLES_INVOLVED', 'NHS':'ON_HIGHWAY', 'DR_DRINK':'DRUNK_DRIVER'}, inplace=True)
df.drop(['HOUR_x'], axis=1, inplace=True)

In [45]:
# create new feature and rename feature
df['SPEED_ABOVE_SPEED_LIMIT'] = df["TRAV_SP"] - df['VSPD_LIM']
df.rename(columns = {'HOUR_x':'HOUR'}, inplace=True)

In [46]:
# create new column (string for drunk_or_sober dummy variable)
df.rename(columns = {'DRUNK_DRIVER':'DRUNK_OR_SOBER'}, inplace=True)
df['DRUNK_DRIVER'] = np.where(df['DRUNK_OR_SOBER'] == 1, 'Drunk', 'Sober')

In [49]:
# select features to be used
df = df[['DRUNK_DRIVER', 'BETWEEN_7PM_AND_7AM', 'ROLLOVER', 'SINGLE_VEHICLE_CRASH', 'PEDS', 'SPEED_ABOVE_SPEED_LIMIT']]

In [50]:
df.head()

,DRUNK_DRIVER,BETWEEN_7PM_AND_7AM,ROLLOVER,SINGLE_VEHICLE_CRASH,PEDS,SPEED_ABOVE_SPEED_LIMIT
0,Drunk,1,0,1,0,0
1,Sober,1,1,1,0,0
2,Drunk,1,1,1,0,25
3,Drunk,1,0,1,0,20
4,Sober,0,1,0,0,-50


In [51]:
# view differences between feature means for drunk/sober
df.groupby(['DRUNK_DRIVER']).mean()

,BETWEEN_7PM_AND_7AM,ROLLOVER,SINGLE_VEHICLE_CRASH,PEDS,SPEED_ABOVE_SPEED_LIMIT
DRUNK_DRIVER,,,,,
Drunk,0.712436,0.287564,0.592389,0.065516,9.389565
Sober,0.381722,0.116291,0.290331,0.176954,-4.494481


In [ ]:
# put df in excel format for tableau to create visuals
df.to_excel('drunkDrivers.xlsx')

In [52]:
# dump df as picklefile
with open('FatalAccidentsEdited', 'wb') as picklefile:
    pkl.dump(df, picklefile)